# Machine learning for Chemistry - tutorial

## Project example 1  - Recurrent Neural networks

In this computer tutorial, we will build a recurrent neural network (RNN) to generate molecular structures. Several topics that we have learned throughout this course will come together in this assignment, including:
* SMILES codes to represent molecules,
* the RDKit library to draw molecular structures,
* the cross-entropy, which we will use for the loss function,
* PyTorch to design the neural network.

The main new concepts that we will learn in this tutorial are: 
1. building, training, and applying a recurrent neural network, 
1. using a neural network for generative modeling.


RNNs are powerful neural network architectures that can deal with sequential data and with time series. In contrast to a vanilla feed-forward neural network in which an input data point, $x_i$, transfers from the first hidden layer to the last hidden layer to result in an output $y_i$, in a recurrent neural network, an input $x_i$ is not only passed from the first hidden layer forward to the next layer, but also backward to work as input to the same hidden unit again. As a result, $x_i$ not only determines $y_i$, but also affects (together with $x_{i+1}$) the next output $y_{i+1}$, and also later output values. The network is said to have _memory_.

A common application for RNNs is in natural language processing, text can be seen as a sequences of sentences, a sentence is a sequence of words, and a word is a sequences of letters. For a given language, there is a relation between the letters in a word, and between the words in a sentence. In other words, given the first three letters of a word, for example "mac", the probability for a possible fourth letters is not a flat distribution over the English alphabet.


In this tutorial, we will use an RNN to learn the pattern in molecules and use that to construct new molecules. This is done using the SMILES representations of the molecules. A SMILES code is a sequence of letters and special tokens, which can be seen as words of a special language with a simple grammar.

We train the network to predict the next character in SMILES strings based on the previous character and the current memory state. The loss function represents a penalty for the incorrect prediction for the next character. When the network is trained enough, we can use it to sample new compounds. For sampling, we just put a <BOS> (begin of sequence) symbol as the initial token, and the network generates the next token. At the next step, we use this token as input and repeat the process to generate the full sequence of tokens. 

Tasks to fulfill:
* build a RNN with pytorch
* learn how to save and load the state of a (partially) trained neural network
* generate new SMILES codes
* visualize the generated molecules


In [2]:
# # first we load some useful libraries
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import one_hot
import functools
from tqdm import tqdm
from rdkit import RDLogger
from rdkit import Chem
from rdkit.Chem import Draw

__special__ = {0: "<PAD>", 1: "<BOS>", 2: "<EOS>"}
RDLogger.DisableLog('rdApp.*')

The class “SmilesProvider” inherits the PyTorch DataLoader class. At initialization, it "tokenizes" all molecules from a dataset and builds a dictionary with indexes for each SMILES token. Here a token is the smallest unit  used to make Smiles "words" and is made from 1 or 2 characters, such as "c", "C", "Si", etc.

In the following cell the “SmilesProvider” class is defined. We will just make of the class as is, e.g. to read the training set. 

$\color{DarkBlue}{\textbf{Task:}}$
* try to understand which <code>file</code> this class reads. This file is provided in the Canvas Module. Download and add this file to the same folder as this jupyter file. Open the file in jupyter lab or in a text editor, to see what is in this file.
* execute the cell


In [3]:
class SmilesProvider(torch.utils.data.DataLoader):
    def __init__(self, file, total=130):
        self.total = total
        self.smiles = open(file, 'r').read().split("\n")[:-1]
        tokens = functools.reduce(lambda acc,s: acc.union(set(s)), self.smiles ,set())
        self.vocsize = len(tokens) + len(__special__)
        self.index2token = dict(enumerate(tokens,start=3))
        self.index2token.update(__special__)
        self.token2index = {v:k for k,v in self.index2token.items()}
        self.ints = [torch.LongTensor([self.token2index[s] for s in line]) for line in tqdm(self.smiles,"Preparing of a dataset")]

    def decode(self,indexes):
        return "".join([self.index2token[index] for index in indexes if index not in __special__])

    def __getitem__(self,i):
        special_added = torch.cat((torch.LongTensor([self.token2index['<BOS>']])
                                   ,self.ints[i],torch.LongTensor([self.token2index['<EOS>']]),
                                   torch.LongTensor([self.token2index["<PAD>"]]*(self.total-len(self.ints[i])-2))),dim=0)
        return one_hot(special_added,self.vocsize).float(),special_added

    def __len__(self):
        return len(self.smiles)
    

Also the following class, we will just use and run as is...

In [13]:
class SmilesLSTM(nn.Module):
    def __init__(self, vocsize, device, max_len=130, hidden_size=256, num_layers=1, num_units=3):
        super().__init__()
        self.device = device
        self.vocsize = vocsize
        self.max_len = max_len
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_units = num_units
        self.lstm_1 = nn.LSTM(vocsize, hidden_size, bidirectional=False, batch_first=True, num_layers=num_layers, dropout = 0.5)
        self.lstm_2 = nn.LSTM(hidden_size, hidden_size, bidirectional=False, batch_first=True, num_layers=num_layers, dropout = 0.3)
        self.linear = nn.Linear(hidden_size, vocsize)

# This doesnt do anaything it just selctects the first caracter and uses that. Also not used?
    def forward(self, x):
        # if hc == None:
        #     h_1 = torch.zeros((self.num_layers, x.size(0), self.hidden_size)).to(self.device)
        #     c_1 = torch.zeros((self.num_layers, x.size(0), self.hidden_size)).to(self.device)
        #     h_2 = torch.zeros((self.num_layers, x.size(0), self.hidden_size)).to(self.device)
        #     c_2 = torch.zeros((self.num_layers, x.size(0), self.hidden_size)).to(self.device)
        # else:
        #     h_1, c_1, h_2, c_2 = hc

        # accumulator = torch.zeros(x.size(0), self.max_len)
        # for i in range(self.max_len):
        x, _ = self.lstm_1(x)
        x, _ = self.lstm_2(x)
        x = self.linear(x)
        # print("before", x.size())
        # x = F.softmax(x, dim=1)
        # print('softmax', x.size())
        # x = torch.multinomial(x,num_samples=1,replacement=True).squeeze(1)
        # print("multinomial", x.size())
        # accumulator[:,i] = x

        return x

    def sample(self,batch_size=128):
        bos_token = [k for k,v in __special__.items() if v == "<BOS>"][0]
        x = torch.LongTensor([bos_token]*batch_size)
        h_1 = torch.zeros((self.num_layers, batch_size, self.hidden_size)).to(self.device)
        c_1 = torch.zeros((self.num_layers, batch_size, self.hidden_size)).to(self.device)
        h_2 = torch.zeros((self.num_layers, batch_size, self.hidden_size)).to(self.device)
        c_2 = torch.zeros((self.num_layers, batch_size, self.hidden_size)).to(self.device)
        accumulator = torch.zeros(batch_size, self.max_len)
        for i in range(self.max_len):

            x = one_hot(x, self.vocsize).float().unsqueeze(1).to(self.device)
            x, (h_1, c_1) = self.lstm_1(x, (h_1, c_1))
            x, (h_2, c_2) = self.lstm_2(x, (h_2, c_2))
            x = self.linear(x).squeeze(1)
            x = F.softmax(x, dim=1)
            x = torch.multinomial(x, num_samples=1,replacement=True).squeeze(1)
            accumulator[:,i] = x
        return accumulator


## Generating Smiles strings

The <code>generate()</code> function generates Smiles strings.

$\color{DarkBlue}{\textbf{Task:}}$
* Use the RDKit function <code>Chem.MolFromSmiles()</code> to check if the produced Smiles is valid (it returns "None" on failure) and increase the variable <code>correct</code> otherwise.
* Set the PyTorch model in the <code>eval</code> modus in the correct place. (Note that this can be anywhere in the cell below)

In [14]:
def generate(file='genmodelLSTM.pt',batch_size=64):
    """
    This is the entrypoint for the generator of SMILES
    :param file: A file with pretrained model
    :param batch_size: The number of compounds to generate
    :return: None. It prints a list of generated compounds to stdout
    """
    box= torch.load(file)
    model,tokenizer = box['model'],box['tokenizer']
    model.eval()
    res = model.sample(batch_size)
    correct = 0
    list_smiles = []
    for i in range(res.size(0)):
        smiles = "".join([tokenizer[index] for index in res[i].tolist() if index not in __special__])
        # print(smiles)
# ======== start your code here =================================
        if Chem.MolFromSmiles(smiles) != None:
            correct += 1
            list_smiles.append(smiles)
# ======== end your code here ===================================
    # print ("% of correct molecules is {:4.2f}".format(correct/float(batch_size)*100))
    return list_smiles, correct/float(batch_size)*100



## Training the RNN

The recurrent neural network learns valid Smiles strings from the training data set of Smiles examples. However, running the training takes a lot of time, een on a fast GPU. 

We will therefore not actually perform the training here. Instead, we will load a trained state of the model and use that to do predictions.

Nevertheless, let's have a look at the training function <code>train</code> below, and fill in a number of blancs in the code.

$\color{DarkBlue}{\textbf{Task 1:}}$
In the relevant section below, add the following:
* set the <code>optimizer</code> variable to the "Adam" optimizer of Pytorch
* set the <code>loss_function</code> variable to a cross-entropy loss function from PyTorch nn module
* switch the PyTorch optimizer in the "train" mode.

$\color{DarkBlue}{\textbf{Task 2:}}$
In the relevant sections below, add in the correct spots:
* set the gradiants of the PyTorch optimizer to zero
* compute the backprop gradients for the loss function
* do a step with the optimizer

In [22]:
def train(file='250k.smi',batch_size=256,learning_rate=0.001,n_epochs=10,device='cuda'):
    """
    This is the entrypoint for training of the RNN
    :param file: A file with molecules in SMILES notation
    :param batch_size: A batch size for training
    :param learning_rate: A learning rate of the optimizer
    :param n_epochs: A number of epochs
    :param device: "cuda" for GPU training, "cpu" for training on CPU, if there are no CUDA on a computer it uses CPU only
    :return: None. It saves the model to "genmodel.pt" file
    """
    device = device if torch.cuda.is_available() else 'cpu'
    dataset = SmilesProvider(file)
    model = SmilesLSTM(dataset.vocsize, device=device).to(device)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
# ======== TASK 1 start your code here =================================
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()
    model.train()
# ======== TASK 1 end your code here ===================================
    for epoch in range(1, n_epochs + 1):
        for iteration,(batch,target) in enumerate(tqdm(dataloader,'Training')):
            batch, target = batch.to(device), target.to(device)
            out = model(batch)
            out = out.transpose(2,1)
            # what does the slcing do it removes the last from prediction and the first carater from the target
            loss = loss_function(out[:,:,:-1],target[:,1:])
            # print('loss', loss)
            # print('batch', batch)
            # print("prediction", out.size())
            # print("prediction", out[0,:,:])

            # print("target", target.size())
            # print("target", target[0,:])

# ======== TASK 2 start your code here =================================
            optimizer.zero_grad()
# ======== TASK 2 end your code here ===================================
            loss.backward()
# ======== TASK 2 start your code here =================================
            optimizer.step()


# ======== TASK 2 end your code here ===================================
    model.device = 'cpu'
    torch.save({'tokenizer':dataset.index2token,'model':model.cpu()},"genmodelLSTM.pt")
train() 


Training: 100%|██████████| 975/975 [01:33<00:00, 10.41it/s]


## Generating molecules

$\color{DarkBlue}{\textbf{Task:}}$
* call the <code>generate()</code> function to produce a list of molecules 

In [38]:
# ======== start your code here =================================
s, c = generate()
print(c, "%")
# ======== end your code here ===================================

90.625 %


## Visualizing molecules

$\color{DarkBlue}{\textbf{Task:}}$
* Use RDKit to draw some of the produced chemical structures to assess how realistic they are.

In [29]:
# ======== start your code here =================================
batch_size = 64
smiles, pc = generate(batch_size=batch_size)
n_print = 9
len_smiles = len(smiles)
indicies = []
mols = []
n = 0
if len_smiles >= n_print:

    while n < len_smiles:
        index = n-1
        if index not in indicies:
            indicies.append(index)
            mols.append(Chem.MolFromSmiles(smiles[index]))
            n += 1

    Draw.MolsToGridImage(mols)
    print(smiles)

# ======== end your code here ===================================

['O=c1[nH]c(-c2ccc(Cl)cc2)nc2sc(N3CCCC3)nc12', 'C[C@@H](NC(=O)N[C@@H]1CCCCC[C@@H]1O)c1ccc(F)c(Cl)c1', 'O=C(Cn1nnc2ccccc2c1=O)N1CCC(c2cccc(C(F)(F)F)c2)CC1', 'Cc1cc(C(=O)NN(C2CCCCC2)[C@@H]2CCS(=O)(=O)C2)ccn1', 'N#C[C@]1(N)CCC[C@@H](Sc2ccc(F)cc2)C1', 'C[C@]1(NC(=O)c2cc3c(s2)CCC3)CCSCC[C@@H]1C(C)(C)C', 'CCc1nc(-c2ccncc2)nc(N2CCCCC2)n1', 'Cc1cccc(CN(C2CC2)[C@@H](CS(C)(=O)=O)c2ccc(F)cc2)c1', 'CCCCOc1ccc(Br)cc1C[NH2+]C(C)C', 'COc1ccc(OC)c(NS(=O)(=O)c2ccc(C(=O)NCc3nccc(-c4ccccc4)c3)s2)c1', 'CC[C@H](Cn1cc[nH+]c1)NC(=O)c1ccoc1C', 'CC[C@@H](Oc1ccc(OC)cc1)C(=O)Nc1cccc([N+](=O)[O-])c1', 'COc1cccc(CNC(=O)C[NH+](C)[C@H]2CCSc3ccccc32)c1', 'Cc1csc([C@@H](C#N)c2ccc([N+](=O)[O-])cc2Cl)n1', 'CC(C)n1nccc1N[C@@H](C)c1nc(Cc2cccs2)no1', 'CCN(CC)C(=O)c1cc(C(=O)N2CCc3c(O)cccc32)sc1C', 'O=c1[nH]ncn1CCN1CCO[C@@H](c2ccc3ccccc3c2)C1', 'CN(C)C(=O)Cc1cnc(NC[C@@H]2CCCO2)s1', 'COc1ccc(NC(=O)Nc2cc(C)ccc2C)c(C)c1', 'COc1ccccc1NC(=O)N1CCN(C[C@@H](C)O)CC1', 'Cn1ncnc1C(=O)[C@H](C#N)c1ccccc1', 'CC[C@@H](C)[NH2+]Cc1cc(F)c(S(N

$\color{DarkBlue}{\textbf{Question 1:}}$ How can the RNN model be further improved to generate useful molecules with certain desired properties?

$\color{Grey}{\textsf{<double-click and type your answer here>}}$

## Conclusions

This ends this part of the final ML4Chem Tutorial on Advanced Neural Networks.

In this tutorial, we have used a RNN to generate new molecules from Smiles strings. Apart from obtaining a deeper understanding of the possibilites of neural networks, the PyTorch library, saving and loading trainined networks, we hope that this tutorial has also inspired you to think of other possibilities and applications of machine learning for chemistry! 
